# Default Rate Sensitivity on Marketing Spend Framework - MX

The effects of changing default rates on Liang & Zu's marketing spend framework is explored.

---

In [1]:
import pandas as pd
import numpy as np
import pyltv
from dbm import DBM
import plotly
from plotly import graph_objects as go
from plotly.subplots import make_subplots
from scipy.optimize import curve_fit

In [2]:
data = pd.read_csv('data/mx_data.csv')

### Study Assumptions
1. Default rate stress applied as a blanket stress across all time periods.
2. Default rate stress applied to both 7dpd and 365dpd default rates.
3. The default rate stress is simply a percentage point added to the actual default rate. **So if the 365dpd default rate is currently 6%, adding a 1% stress results in a new 365dpd default rate of 7%**.

## Impact of Default Rate on LTV

In [36]:
market='mx'
forecast_method='powerslope'
min_months=4
n_months=50
default_stress=0.01

m1 = pyltv.Model(data, market=market, fcast_method=forecast_method)
m1.generate_features()
m1.forecast = m1.forecast_data(m1.data, min_months, n_months)

m2 = pyltv.Model(data, market=market, fcast_method=forecast_method, default_stress=default_stress)
m2.generate_features()
m2.forecast = m2.forecast_data(m2.data, min_months, n_months)

Data spans 2020-09 to 2022-01
Total # of cohorts: 17
...
Data spans 2020-09 to 2022-01
Total # of cohorts: 17
...


In [37]:
colors = {i: c for i, c in enumerate(plotly.colors.qualitative.Dark24)}
# add additional colors
color_size = len(colors)
for i, c in enumerate(plotly.colors.qualitative.Light24):
    colors[i+color_size] = c

fig = make_subplots(rows=1, cols=2, shared_xaxes=True,
                   subplot_titles=('Unstressed', '+1% Default Rates'))

param = 'cumulative_dcf_ltv_per_original'

# LEFT PLOT
curves1=[]
for cohort in m1.forecast.cohort.unique():
    c_data = m1.forecast[m1.forecast.cohort == cohort]
    for dtype in c_data.data_type.unique():
        output = c_data[c_data.data_type == dtype][param]

        output.name = cohort + '-' + dtype

        curves1.append(output)
    
for i, cohort in enumerate(curves1):
    c = colors[i]
    if 'forecast' in cohort.name:
        fig.add_trace(go.Scatter(name=cohort.name, x=cohort.index, y=cohort, mode='lines',
                                 line=dict(width=3, dash='dash', color=c), legendgroup=f'{i}'), row=1, col=1)
    else:
        if cohort.notnull().any():
            fig.add_trace(go.Scatter(name=cohort.name, x=cohort.index, y=cohort, mode='markers+lines',
                                     line=dict(width=2, color=c), legendgroup=f'{i}'), row=1, col=1)

expected = m1.ltv_expected.cumulative_ltv_per_original

fig.add_trace(go.Scatter(name='expected', x=expected.index, y=expected, 
                         line=dict(color='black', width=3), legendgroup='e'), row=1, col=1)
fig.add_trace(go.Scatter(name='expected', x=expected.index, y=expected, 
                        line=dict(color='black', width=3), legendgroup='e', showlegend=False), row=1, col=2)

# RIGHT PLOT
curves2=[]
for cohort in m2.forecast.cohort.unique():
    c_data = m2.forecast[m2.forecast.cohort == cohort]
    for dtype in c_data.data_type.unique():
        output = c_data[c_data.data_type == dtype][param]

        output.name = cohort + '-' + dtype

        curves2.append(output)
        
for i, cohort in enumerate(curves2):
    c = colors[i]
    if 'forecast' in cohort.name:
        fig.add_trace(go.Scatter(name=cohort.name, x=cohort.index, y=cohort, mode='lines',
                                 line=dict(width=3, dash='dash', color=c), legendgroup=f'{i}', showlegend=False), row=1, col=2,)
    else:
        if cohort.notnull().any():
            fig.add_trace(go.Scatter(name=cohort.name, x=cohort.index, y=cohort, mode='markers+lines',
                                     line=dict(width=2, color=c), legendgroup=f'{i}', showlegend=False), row=1, col=2)

fig.update_layout(title='DCF LTV')
fig.layout.xaxis.title='Month'
fig.layout.xaxis2.title='Month'
fig.update_yaxes(range=[-10, 95])
fig.update_xaxes(range=[-5,52])

fig.show()

### Standard Default Rate

In [63]:
data_50mo1 = m1.forecast[m1.forecast['Months Since First Loan Disbursed']==50]
ltv1 = data_50mo1['cumulative_ltv_per_original'].median()
dcf_ltv1 = data_50mo1['cumulative_dcf_ltv_per_original'].median()
#print(f'50mo LTV: ${round(ltv1, 2)}')
print(f'Median DCF 50mo LTV: ${round(dcf_ltv1, 2)}')

Median DCF 50mo LTV: $64.37


### +1% Default Rate Stress

In [64]:
data_50mo2 = m2.forecast[m2.forecast['Months Since First Loan Disbursed']==50]
ltv2 = data_50mo2['cumulative_ltv_per_original'].median()
dcf_ltv2 = data_50mo2['cumulative_dcf_ltv_per_original'].median()
#print(f'50mo LTV: ${round(ltv2, 2)}')
print(f'Median DCF 50mo LTV: ${round(dcf_ltv2, 2)}')

Median DCF 50mo LTV: $53.98


In [65]:
print('Change in DCF LTV:')
print(f'{round(100*(dcf_ltv2-dcf_ltv1)/dcf_ltv1, 2)}%')

Change in DCF LTV:
-16.14%


In [66]:
ltvs = {}
for x in np.linspace(-0.03, 0.03, 7):
    m = pyltv.Model(data, market=market, fcast_method=forecast_method, default_stress=x)
    m.generate_features()
    m.forecast = m.forecast_data(m.data, min_months, n_months)
    
    data_50mo = m.forecast[m.forecast['Months Since First Loan Disbursed']==50]
    m.forecast['defaulted_origination'] = m.forecast['default_rate_365dpd']*m.forecast.origination_per_original
    default = m.forecast.defaulted_origination.sum()/m.forecast.origination_per_original.sum()
    
    ltvs[round(x, 2)] = (round(data_50mo['cumulative_dcf_ltv_per_original'].median(), 2), 
                         round(100*default, 2))

Data spans 2020-09 to 2022-01
Total # of cohorts: 17
...
Data spans 2020-09 to 2022-01
Total # of cohorts: 17
...
Data spans 2020-09 to 2022-01
Total # of cohorts: 17
...
Data spans 2020-09 to 2022-01
Total # of cohorts: 17
...
Data spans 2020-09 to 2022-01
Total # of cohorts: 17
...
Data spans 2020-09 to 2022-01
Total # of cohorts: 17
...
Data spans 2020-09 to 2022-01
Total # of cohorts: 17
...


### Default Rate Stress: -3% to +3%

The plot below shows the relationship between a change in the default rate stress and the median DCF LTV for all cohorts.

In [67]:
v = [i[0] for i in ltvs.values()]
l = [f'{i[1]}%' for i in ltvs.values()]
fig = go.Figure(go.Scatter(x=l, y=v))
fig.update_layout(xaxis=dict(title='Default Rate Stress'),
                 yaxis=dict(title='Median DCF LTV'))
fig.show()

In [68]:
print('For every +1% increase in default rate...')
print(f'-${round(ltvs[0][0]-ltvs[0.01][0], 2)}')

For every +1% increase in default rate...
-$10.39


The relationships between LTV and default rates is shown below and explains the linear trend. **Note that this analysis does not include the effects of default rate changes on retention.**

LTV ~ [revenue - (origination+revenue)*default_rate_365dpd]

revenue ~ [0.08 * default_rate_7dpd]

## Default Rate Historicals

#### Weighted Average Lifetime Default

This method uses the average lifetime default rate (365dpd) for each cohort. The average lifetime default rate is a weighted average of defaults by origination_per_original.

In [69]:
defaults = []
default_changes = []
avg_lifetime_default = []
for c in m1.forecast.cohort.unique():
    c_data = m1.forecast[m1.forecast.cohort==c][['origination_per_original', 'default_rate_365dpd']]
    c_data['defaulted_origination'] = c_data['default_rate_365dpd']*c_data.origination_per_original
    c_data['defaulted_origination_change'] = (c_data.defaulted_origination-c_data.defaulted_origination.shift(1))/ \
        c_data.defaulted_origination
    
    defaults.append(go.Scatter(name=c, x=c_data.index, y=c_data.defaulted_origination))
    default_changes.append(go.Scatter(name=c, x=c_data.index, y=c_data.defaulted_origination_change))
    
    avg_lifetime_default.append(c_data.defaulted_origination.sum()/c_data.origination_per_original.sum())
    
avg_lifetime_default = pd.Series(avg_lifetime_default, index=m1.forecast.cohort.unique())

In [70]:
fig=go.Figure(defaults)
fig.update_layout(xaxis=dict(title='Month'), yaxis=dict(title='Defaulted Origination per Original'))
fig.show()

In [71]:
fig = go.Figure(
    go.Scatter(name=c, x=avg_lifetime_default.index, y=avg_lifetime_default, mode='markers',
                              marker=dict(size=15))
)

fig.update_layout(xaxis=dict(title='Month'), yaxis=dict(title='Default Rate'))
fig.show()

In MX, the default rates are much higher on average. The median weighted average default rate is 9.1%. There is some apparent seasonality in the default rates that's not seen in the other markets. **We see a maximum weighted average default rate of 10.7% for MX**.

#### Unweighted Average Default

In this method, we simply look at aggregate stats of the 365 dpd default rate, without taking into account origination. This scenario looks at a blanket default rate increase across the board, regardless of the size of loans.

In [72]:
default_stats = {}
for i, c in enumerate(m1.forecast.cohort.unique()):
    c_data = m1.forecast[m1.forecast.cohort==c][['default_rate_365dpd']]
    
    default_stats[c] = (float(c_data.median()), float(c_data.quantile(0.9)))

In [73]:
medians=[]
quantile90=[]
for c in default_stats:
    medians.append(default_stats[c][0])
    quantile90.append(default_stats[c][1])
    
medians = pd.Series(medians, index=np.arange(1, len(medians)+1))
#params1, covs1 = curve_fit(reg, xdata=medians.index, ydata=medians)

quantile90 = pd.Series(quantile90, index=np.arange(1, len(medians)+1))
#params2, covs2 = curve_fit(reg, xdata=quantile90.index, ydata=quantile90)
    
fig = go.Figure([
    go.Scatter(name='median default', x=medians.index, y=medians, mode='markers'),
    #go.Scatter(name='median-fit', x=np.arange(1,36), y=reg(np.arange(1,36), params1[0], params1[1])),
    go.Scatter(name='90 percentile', x=quantile90.index, y=quantile90, mode='markers')
    #go.Scatter(name='90 percentile-fit', x=np.arange(1,36), y=reg(np.arange(1,36), params2[0], params2[1]))
])

fig.update_layout(xaxis=dict(title='Month'), yaxis=dict(title='Default Rate'))
fig.show()

Similarly, in looking at both the 50% and 90% quantiles of the 365dpd default rates, the trend is quite flat with some similar seasonality as the weighted average default rates. The max we see here for MX is 12.45%.

## Payback Period

### mCAC Payback Period

In [104]:
ratios = [1, 1.5, 2]
paybacks = []
for c in m1.forecast.cohort.unique():
    c_data = m1.forecast[m1.forecast.cohort==c]
    
    dcf_ltv = float(c_data[c_data['Months Since First Loan Disbursed']==50]['cumulative_dcf_ltv_per_original'])
    
    record = {'cohort': c, 'DCF LTV': dcf_ltv}
    for r in ratios:
        mcac = float(dcf_ltv/r)
        if r == 2:
            cac_ratio = 4.41
        elif r == 1.5:
            cac_ratio = 3.73
        elif r == 1:
            cac_ratio = 2.73
            
        avg_cac = float(dcf_ltv/cac_ratio)

        dcf = c_data['cumulative_dcf_ltv_per_original']
        payback_idx = dcf[dcf >= mcac].index[0]

        record[f'mCAC ({r})'] = mcac
        record[f'CAC ({r})'] = avg_cac
        record[f'Payback Period ({r})'] = payback_idx
        record[f'Payback Amount ({r})'] = dcf.loc[payback_idx]
    
    paybacks.append(record)
    
payback_results1 = pd.DataFrame.from_records(paybacks)

In [105]:
ratios = [1.5, 2]
paybacks = []
for c in m1.forecast.cohort.unique():
    c_data = m1.forecast[m1.forecast.cohort==c]
    c_data_stressed = m2.forecast[m2.forecast.cohort==c]
    
    dcf_ltv = float(c_data[c_data['Months Since First Loan Disbursed']==50]['cumulative_dcf_ltv_per_original'])
    dcf_ltv_stressed = float(c_data_stressed[c_data_stressed['Months Since First Loan Disbursed']==50]['cumulative_dcf_ltv_per_original'])
    
    record = {'cohort': c, 'DCF LTV': dcf_ltv, 'DCF LTV Stressed': dcf_ltv_stressed}
    for r in ratios:
        mcac = float(dcf_ltv/r)
        if r == 2:
            cac_ratio = 4.41
        elif r == 1.5:
            cac_ratio = 3.73
        elif r == 1:
            cac_ratio = 2.73
            
        avg_cac = float(dcf_ltv/cac_ratio)
        
        dcf = c_data_stressed['cumulative_dcf_ltv_per_original']
        
        payback_idx = dcf[dcf >= mcac].index[0]

        record[f'mCAC ({r})'] = mcac
        record[f'CAC ({r})'] = avg_cac
        record[f'Payback Period ({r})'] = payback_idx
        record[f'Payback Amount ({r})'] = dcf.loc[payback_idx]
    
    paybacks.append(record)
    
payback_results2 = pd.DataFrame.from_records(paybacks)

The table below shows the payback periods for unstressed default rates in the 1.5x scenario.

In [106]:
ratio = 1.5
payback_results1[['cohort', 'DCF LTV', f'mCAC ({ratio})', f'Payback Period ({ratio})', f'Payback Amount ({ratio})']]

,cohort,DCF LTV,mCAC (1.5),Payback Period (1.5),Payback Amount (1.5)
0,2020-09,66.310969,44.207313,22,45.639603
1,2020-10,64.514343,43.009562,22,43.342011
2,2020-11,54.779790,36.519860,23,36.698495
3,2020-12,39.889508,26.593005,25,27.103509
4,2021-01,35.668625,23.779083,26,24.403940
5,2021-02,36.591121,24.394080,26,24.941443
6,2021-03,47.484118,31.656079,25,32.240410
7,2021-04,64.221207,42.814138,24,43.377615
8,2021-05,87.477536,58.318357,24,60.200300
9,2021-06,85.281141,56.854094,24,58.419400


The second table shows the new payback periods in the same 1.5x scenario, but with a +1% default rate stress applied. The payback periods are pushed out by several months.

In [107]:
ratio = 1.5
payback_results2[['cohort', 'DCF LTV', 'DCF LTV Stressed', f'mCAC ({ratio})', f'Payback Period ({ratio})', f'Payback Amount ({ratio})']]

,cohort,DCF LTV,DCF LTV Stressed,mCAC (1.5),Payback Period (1.5),Payback Amount (1.5)
0,2020-09,66.310969,55.642429,44.207313,29,44.972187
1,2020-10,64.514343,54.125290,43.009562,29,43.159456
2,2020-11,54.779790,44.922738,36.519860,32,37.197913
3,2020-12,39.889508,31.415716,26.593005,35,26.667759
4,2021-01,35.668625,27.654273,23.779083,37,23.991136
5,2021-02,36.591121,28.602445,24.394080,37,24.816243
6,2021-03,47.484118,38.638892,31.656079,33,31.728184
7,2021-04,64.221207,53.833467,42.814138,31,43.308810
8,2021-05,87.477536,74.964890,58.318357,29,58.587511
9,2021-06,85.281141,72.845063,56.854094,30,58.034669


In [108]:
fig1 = go.Figure([go.Scatter(name='1x', x=payback_results1['Payback Period (1)'], 
                            y=payback_results1['Payback Amount (1)'], mode='markers',
                            marker=dict(color='red')),
                  go.Scatter(name='1.5x', x=payback_results1['Payback Period (1.5)'], 
                            y=payback_results1['Payback Amount (1.5)'], mode='markers',
                            marker=dict(color='blue')),
                 go.Scatter(name='2x', x=payback_results1['Payback Period (2)'], 
                            y=payback_results1['Payback Amount (2)'], mode='markers',
                            marker=dict(color='orange'))
                ])

fig1.update_layout(title='Unstressed Default Rate')
fig1.update_layout(xaxis=dict(title='Month', range=[12, 40]), yaxis=dict(title='DCF LTV'))

fig2 = go.Figure([go.Scatter(name='1.5x', x=payback_results2['Payback Period (1.5)'], 
                            y=payback_results2['Payback Amount (1.5)'], mode='markers',
                            marker=dict(color='blue')),
                 go.Scatter(name='2x', x=payback_results2['Payback Period (2)'], 
                            y=payback_results2['Payback Amount (2)'], mode='markers',
                            marker=dict(color='orange'))
                ])

fig2.update_layout(title='+1% Default Rate')
fig2.update_layout(xaxis=dict(title='Month', range=[12, 40]), yaxis=dict(title='DCF LTV'))

fig1.show()
fig2.show()

In [109]:
payback_results2

,cohort,DCF LTV,DCF LTV Stressed,mCAC (1.5),CAC (1.5),Payback Period (1.5),Payback Amount (1.5),mCAC (2),CAC (2),Payback Period (2),Payback Amount (2)
0,2020-09,66.310969,55.642429,44.207313,17.777740,29,44.972187,33.155485,15.036501,19,33.345043
1,2020-10,64.514343,54.125290,43.009562,17.296070,29,43.159456,32.257171,14.629103,20,32.686478
2,2020-11,54.779790,44.922738,36.519860,14.686271,32,37.197913,27.389895,12.421721,22,27.951939
3,2020-12,39.889508,31.415716,26.593005,10.694238,35,26.667759,19.944754,9.045240,25,20.276505
4,2021-01,35.668625,27.654273,23.779083,9.562634,37,23.991136,17.834313,8.088124,26,17.836640
5,2021-02,36.591121,28.602445,24.394080,9.809952,37,24.816243,18.295560,8.297306,26,18.447889
6,2021-03,47.484118,38.638892,31.656079,12.730327,33,31.728184,23.742059,10.767374,24,24.223631
7,2021-04,64.221207,53.833467,42.814138,17.217482,31,43.308810,32.110604,14.562632,22,32.606920
8,2021-05,87.477536,74.964890,58.318357,23.452422,29,58.587511,43.738768,19.836176,21,44.939207
9,2021-06,85.281141,72.845063,56.854094,22.863577,30,58.034669,42.640571,19.338127,21,43.354267


In [86]:
payback_results1['Payback Period (1.5)'] - payback_results2['Payback Period (1.5)']

0     -7
1     -7
2     -9
3    -10
4    -11
5    -11
6     -8
7     -7
8     -5
9     -6
10    -6
11    -6
Name: Payback Period (1.5), dtype: int64

In the worst case, the **payback period will shift out 11 months at an LTV/mCAC of 1.5**, and by **6 months for an LTV/mCAC of 2**.

### Average CAC Payback Period

In [87]:
ratios = [1, 1.5, 2]
paybacks = []
for c in m1.forecast.cohort.unique():
    c_data = m1.forecast[m1.forecast.cohort==c]
    
    dcf_ltv = float(c_data[c_data['Months Since First Loan Disbursed']==50]['cumulative_dcf_ltv_per_original'])
    
    record = {'cohort': c, 'DCF LTV': dcf_ltv}
    for r in ratios:
        mcac = float(dcf_ltv/r)
        if r == 2:
            cac_ratio = 4.41
        elif r == 1.5:
            cac_ratio = 3.73
        elif r == 1:
            cac_ratio = 2.73
            
        avg_cac = float(dcf_ltv/cac_ratio)

        dcf = c_data['cumulative_dcf_ltv_per_original']
        payback_idx = dcf[dcf >= avg_cac].index[0]

        record[f'mCAC ({r})'] = mcac
        record[f'CAC ({r})'] = avg_cac
        record[f'Payback Period ({r})'] = payback_idx
        record[f'Payback Amount ({r})'] = dcf.loc[payback_idx]
    
    paybacks.append(record)
    
payback_results1 = pd.DataFrame.from_records(paybacks)

In [88]:
ratios = [1, 1.5, 2]
paybacks = []
for c in m1.forecast.cohort.unique():
    c_data = m1.forecast[m1.forecast.cohort==c]
    c_data_stressed = m2.forecast[m2.forecast.cohort==c]
    
    dcf_ltv = float(c_data[c_data['Months Since First Loan Disbursed']==50]['cumulative_dcf_ltv_per_original'])
    dcf_ltv_stressed = float(c_data_stressed[c_data_stressed['Months Since First Loan Disbursed']==50]['cumulative_dcf_ltv_per_original'])
    
    record = {'cohort': c, 'DCF LTV': dcf_ltv, 'DCF LTV Stressed': dcf_ltv_stressed}
    for r in ratios:
        mcac = float(dcf_ltv/r)
        if r == 2:
            cac_ratio = 4.41
        elif r == 1.5:
            cac_ratio = 3.73
        elif r == 1:
            cac_ratio = 2.73
            
        avg_cac = float(dcf_ltv/cac_ratio)
        
        dcf = c_data_stressed['cumulative_dcf_ltv_per_original']
        
        payback_idx = dcf[dcf >= avg_cac].index[0]

        record[f'mCAC ({r})'] = mcac
        record[f'CAC ({r})'] = avg_cac
        record[f'Payback Period ({r})'] = payback_idx
        record[f'Payback Amount ({r})'] = dcf.loc[payback_idx]
    
    paybacks.append(record)
    
payback_results2 = pd.DataFrame.from_records(paybacks)

The two tables below show the changes in payback periods when using LTV/CAC instead of LTV/mCAC as the guard rail. The shifts in payback periods are much less significant.

In [89]:
ratio = 1.5
payback_results1[['cohort', 'DCF LTV', f'CAC ({ratio})', f'Payback Period ({ratio})', f'Payback Amount ({ratio})']]

,cohort,DCF LTV,CAC (1.5),Payback Period (1.5),Payback Amount (1.5)
0,2020-09,66.310969,17.777740,8,18.018263
1,2020-10,64.514343,17.296070,9,17.831170
2,2020-11,54.779790,14.686271,11,16.310468
3,2020-12,39.889508,10.694238,12,10.728946
4,2021-01,35.668625,9.562634,14,10.823308
5,2021-02,36.591121,9.809952,14,10.867670
6,2021-03,47.484118,12.730327,13,14.264131
7,2021-04,64.221207,17.217482,12,19.429951
8,2021-05,87.477536,23.452422,11,25.143700
9,2021-06,85.281141,22.863577,11,24.097625


In [90]:
ratio = 1.5
payback_results2[['cohort', 'DCF LTV', 'DCF LTV Stressed', f'CAC ({ratio})', f'Payback Period ({ratio})', f'Payback Amount ({ratio})']]

,cohort,DCF LTV,DCF LTV Stressed,CAC (1.5),Payback Period (1.5),Payback Amount (1.5)
0,2020-09,66.310969,55.642429,17.777740,11,19.070726
1,2020-10,64.514343,54.125290,17.296070,12,18.537985
2,2020-11,54.779790,44.922738,14.686271,13,15.293489
3,2020-12,39.889508,31.415716,10.694238,16,11.101816
4,2021-01,35.668625,27.654273,9.562634,18,10.529557
5,2021-02,36.591121,28.602445,9.809952,18,10.876702
6,2021-03,47.484118,38.638892,12.730327,16,14.115448
7,2021-04,64.221207,53.833467,17.217482,14,18.511179
8,2021-05,87.477536,74.964890,23.452422,13,25.325574
9,2021-06,85.281141,72.845063,22.863577,13,24.208471


In [94]:
fig1 = go.Figure([go.Scatter(name='1x', x=payback_results1['Payback Period (1)'], 
                            y=payback_results1['Payback Amount (1)'], mode='markers'),
                  go.Scatter(name='1.5x', x=payback_results1['Payback Period (1.5)'], 
                            y=payback_results1['Payback Amount (1.5)'], mode='markers'),
                 go.Scatter(name='2x', x=payback_results1['Payback Period (2)'], 
                            y=payback_results1['Payback Amount (2)'], mode='markers')
                ])

fig1.update_layout(title='Unstressed Default Rate')
fig1.update_layout(xaxis=dict(title='Month', range=[5, 23]), yaxis=dict(title='DCF LTV'))

fig2 = go.Figure([go.Scatter(name='1x', x=payback_results2['Payback Period (1)'], 
                            y=payback_results2['Payback Amount (1)'], mode='markers'),
                  go.Scatter(name='1.5x', x=payback_results2['Payback Period (1.5)'], 
                            y=payback_results2['Payback Amount (1.5)'], mode='markers'),
                 go.Scatter(name='2x', x=payback_results2['Payback Period (2)'], 
                            y=payback_results2['Payback Amount (2)'], mode='markers')
                ])

fig2.update_layout(title='+1% Default Rate')
fig2.update_layout(xaxis=dict(title='Month', range=[5, 23]), yaxis=dict(title='DCF LTV'))

fig1.show()
fig2.show()

In [96]:
payback_results1['Payback Period (2)'] - payback_results2['Payback Period (2)']

0    -2
1    -2
2    -3
3    -3
4    -3
5    -3
6    -2
7    -3
8    -2
9    -2
10   -2
11   -2
Name: Payback Period (2), dtype: int64

In the worst case, our most recent cohort, the **payback period will shift out by 4 months at an LTV/CAC of 1.5x**, and by **3 for an LTV/CAC of 2x**.

## Conclusions

- A rise in default rate leads to an increas in payback period. This is true for any scenario (1x, 1.5x, 2x). In general, the shift in payback period is worse for lower multipliers (less margin).
- A +1% increase in default rate in the 1.5x scenario will shift the payback period by up to 11 months. At 2x, there is only a shift in payback period of +2 months.
- With a 1.5x scenario, the data suggests we can still maintain a 50mo positive LTV inspite of up to a +1% rise in default rates.